In [27]:
# if you can make this work, do for everything and upload project

In [ ]:
import numpy as np 
import re

def extract_metadata(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Find the header section
    match = re.search(r'begin_of_head[\s=]*([\s\S]*?)end_of_head', content)
    if not match:
        raise ValueError("Header section not found in the file.")
    
    header_content = match.group(1).strip()
    
    # Extract key-value pairs
    metadata = {}
    for line in header_content.splitlines():
        line = line.strip()
        if not line:
            continue
        
        # Handle key-value pairs with colon or equals sign
        match = re.match(r'([\w\s]+)[:=]\s*(.*)', line)
        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()
            metadata[key] = value
    
    return metadata



In [10]:
fn = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\igs\Brazil_2015_MAPGEO2015_gravG_20201222.isg"
tif = fn.replace('.isg','.tif')

In [13]:
import re
import numpy as np

def extract_metadata_and_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Find the header section
    match = re.search(r'begin_of_head[\s=]*([\s\S]*?)end_of_head', content)
    if not match:
        raise ValueError("Header section not found in the file.")
    
    header_content = match.group(1).strip()
    
    # Extract key-value pairs
    metadata = {}
    for line in header_content.splitlines():
        line = line.strip()
        if not line:
            continue
        
        # Handle key-value pairs with colon or equals sign
        match = re.match(r'([\w\s]+)[:=]\s*(.*)', line)
        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()
            metadata[key] = value
    
    # Extract numerical data after the header
    data_start = match.end()
    data_content = content[data_start:].strip()
    
    # Process data while ignoring non-numeric lines
    data_rows = []
    for row in data_content.splitlines():
        row = row.strip()
        if not row:
            continue
        try:
            data_rows.append(list(map(float, row.split())))
        except ValueError:
            continue  # Ignore lines that can't be converted to float
    
    data_array = np.array(data_rows)
    
    return metadata, data_array

# Example usage
filename = fn  # Change this to your actual file
metadata_dict, data_array = extract_metadata_and_data(filename)
print(metadata_dict)
print(data_array.shape)


{'model name': 'MAPGEO2015', 'model year': '2015', 'model type': 'gravimetric', 'data type': 'geoid', 'data format': 'grid', 'data units': 'meters', 'data ordering': 'N-to-S, W-to-E', 'ref ellipsoid': 'GRS80', 'ref frame': 'SIRGAS 2000', 'height datum': '---', 'tide system': '---', 'coord type': 'geodetic', 'coord units': 'dms', 'map projection': '---', 'EPSG code': '4989', 'lat min': '-34°57\'30"', 'lat max': '5°57\'30"', 'lon min': '-74°57\'30"', 'lon max': '-30°02\'30"', 'delta lat': '05\'00"', 'delta lon': '05\'00"', 'nrows': '492', 'ncols': '540', 'nodata': '-9999.0000', 'creation date': '22/12/2020', 'ISG format': '2.0'}
(492, 540)


In [23]:
metadata = metadata_dict.copy()

In [24]:
import numpy as np
import rasterio
from rasterio.transform import from_origin

# Step 1: Parse the metadata


# Step 2: Convert DMS to Decimal Degrees
def dms_to_dd(dms):
    """Convert degrees, minutes, seconds to decimal degrees."""
    parts = dms.split('°')
    degrees = float(parts[0])
    minutes = float(parts[1].split('\'')[0]) / 60.0
    seconds = float(parts[1].split('\'')[1].strip('"')) / 3600.0
    return degrees + minutes + seconds if degrees >= 0 else degrees - minutes - seconds

lat_min = dms_to_dd(metadata['lat min'])
lat_max = dms_to_dd(metadata['lat max'])
lon_min = dms_to_dd(metadata['lon min'])
lon_max = dms_to_dd(metadata['lon max'])

# Step 3: Define resolution in decimal degrees
delta_lat = 5 / 60.0  # 5 minutes in degrees
delta_lon = 5 / 60.0  # 5 minutes in degrees

# Step 4: Define the affine transformation
transform = rasterio.transform.from_origin(
    west=lon_min,
    north=lat_max,
    xsize=delta_lon,
    ysize=delta_lat
)

# Step 5: Prepare the data array


# Step 6: Write the raster file
output_file = "output_raster.tif"
with rasterio.open(
    output_file,
    'w',
    driver='GTiff',
    height=int(metadata['nrows']),
    width=int(metadata['ncols']),
    count=1,  # Single band
    dtype=data_array.dtype,
    crs=f"EPSG:{metadata['EPSG code']}",
    transform=transform,
    nodata=float(metadata['nodata'])
) as dst:
    dst.write(data_array, 1)

print(f"Raster file saved as {output_file}")

Raster file saved as output_raster.tif


In [28]:
fn = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\igs\GEOID_FFT_20190703.isg.txt"

In [29]:
filename = fn  # Change this to your actual file
metadata_dict, data_array = extract_metadata_and_data(filename)
print(metadata_dict)
print(data_array.shape)

{'model name': 'GEOID_FFT', 'model type': 'gravimetric', 'units': 'meters', 'reference': 'WGS84', 'lat min': '7.958333331676', 'lat max': '24.041666665324', 'lon min': '101.958333331686', 'lon max': '110.041666665314', 'delta lat': '0.083333330000', 'delta lon': '0.083333330000', 'nrows': '193', 'ncols': '97', 'nodata': '-9999.0000', 'ISG format': '1.01'}
(193, 97)


In [32]:
fn = r"c:\\Users\\Joseph\\Downloads\\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\\igs\\GEOID_FFT_20190703.isg.txt"

In [34]:
from pprint import pprint

In [36]:
fn = r"c:\Users\Joseph\Downloads\Topo_DEM_Mekong_delta_excl_rivers_and_bedrock\igs\Brazil_2015_MAPGEO2015_gravG_20201222.isg"
tif = fn.replace('.isg','.tif')
filename = fn  # Change this to your actual file
metadata_dict, data_array = extract_metadata_and_data(filename)
pprint(metadata_dict)
print(data_array.shape)

{'EPSG code': '4989',
 'ISG format': '2.0',
 'coord type': 'geodetic',
 'coord units': 'dms',
 'creation date': '22/12/2020',
 'data format': 'grid',
 'data ordering': 'N-to-S, W-to-E',
 'data type': 'geoid',
 'data units': 'meters',
 'delta lat': '05\'00"',
 'delta lon': '05\'00"',
 'height datum': '---',
 'lat max': '5°57\'30"',
 'lat min': '-34°57\'30"',
 'lon max': '-30°02\'30"',
 'lon min': '-74°57\'30"',
 'map projection': '---',
 'model name': 'MAPGEO2015',
 'model type': 'gravimetric',
 'model year': '2015',
 'ncols': '540',
 'nodata': '-9999.0000',
 'nrows': '492',
 'ref ellipsoid': 'GRS80',
 'ref frame': 'SIRGAS 2000',
 'tide system': '---'}
(492, 540)


In [35]:
filename = fn  # Change this to your actual file
metadata_dict, data_array = extract_metadata_and_data(filename)
pprint(metadata_dict)
print(data_array.shape)

{'ISG format': '1.01',
 'delta lat': '0.083333330000',
 'delta lon': '0.083333330000',
 'lat max': '24.041666665324',
 'lat min': '7.958333331676',
 'lon max': '110.041666665314',
 'lon min': '101.958333331686',
 'model name': 'GEOID_FFT',
 'model type': 'gravimetric',
 'ncols': '97',
 'nodata': '-9999.0000',
 'nrows': '193',
 'reference': 'WGS84',
 'units': 'meters'}
(193, 97)


In [38]:
# should be writing this metadata into text file on on raster?
# try both

In [40]:
def isg2tif_f1(isg_fpath,output_file):
    metadata, data_array = extract_metadata_and_data(isg_fpath)
    delta_lat = float(metadata['delta lat'])  # Resolution in latitude (decimal degrees)
    delta_lon = float(metadata['delta lon'])  # Resolution in longitude (decimal degrees)
    lat_min = float(metadata['lat min'])      # Minimum latitude (decimal degrees)
    lat_max = float(metadata['lat max'])      # Maximum latitude (decimal degrees)
    lon_min = float(metadata['lon min'])      # Minimum longitude (decimal degrees)
    lon_max = float(metadata['lon max'])      # Maximum longitude (decimal degrees)
    nrows = int(metadata['nrows'])            # Number of rows
    ncols = int(metadata['ncols'])            # Number of columns
    nodata = float(metadata['nodata'])        # No-data value

    if metadata['reference'] == 'WGS84':
        crs = "EPSG:4326"                         # CRS for WGS84

    transform = rasterio.transform.from_origin(
                    west=lon_min,
                    north=lat_max,
                    xsize=delta_lon,
                    ysize=delta_lat
                )
    with rasterio.open(
                        output_file,
                        'w',
                        driver='GTiff',
                        height=nrows,
                        width=ncols,
                        count=1,  # Single band
                        dtype=data_array.dtype,
                        crs=crs,
                        transform=transform,
                        nodata=nodata
                    ) as dst:
                        dst.write(data_array, 1)

    print(f"Raster file saved as {output_file}")


In [41]:
filename = fn  # Change this to your actual file
metadata_dict, data_array = extract_metadata_and_data(filename)
pprint(metadata_dict)
print(data_array.shape)

{'EPSG code': '4989',
 'ISG format': '2.0',
 'coord type': 'geodetic',
 'coord units': 'dms',
 'creation date': '22/12/2020',
 'data format': 'grid',
 'data ordering': 'N-to-S, W-to-E',
 'data type': 'geoid',
 'data units': 'meters',
 'delta lat': '05\'00"',
 'delta lon': '05\'00"',
 'height datum': '---',
 'lat max': '5°57\'30"',
 'lat min': '-34°57\'30"',
 'lon max': '-30°02\'30"',
 'lon min': '-74°57\'30"',
 'map projection': '---',
 'model name': 'MAPGEO2015',
 'model type': 'gravimetric',
 'model year': '2015',
 'ncols': '540',
 'nodata': '-9999.0000',
 'nrows': '492',
 'ref ellipsoid': 'GRS80',
 'ref frame': 'SIRGAS 2000',
 'tide system': '---'}
(492, 540)


In [45]:
import pandas as pd 

In [46]:
html = "https://www.isgeoid.polimi.it/Geoid/reg_list.html"

In [47]:
df = pd.read_html(html)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>